# Векторные представления слов

# Векторные представления FastText
В предыдущих представлениях слово должно быть в словаре, чтобы можно было найти для него вектор. Но что делать, если слова нет в словаре?

Зададимся вопросом, что такое слово?

В векторном представлении "слово" это элемент словаря, а значит им может быть все что угодно: смайлики, картинки, знаки пунктуации, словосочетания и т.п.

Набор букв (символов) тоже может быть элементом словаря. А давайте все тексты разобьем на последовательности символов определенной длинны (их называют n-граммы) и будем создавать вектора для этих последовательностей.

Реальное слово можно представить несколькими такими n-граммами, объединим (сложим) вектора для них и получим вектор для настоящего слова.

Никто не запрещает иметь в словаре одновременно и настоящие слова и их n-граммы.

Но это значит что для любого слова, которого даже нет в словаре, мы можем построить вектор из векторов его n-грамм. Такой подход реализован, например, в [FastText](https://amitness.com/2020/06/fasttext-embeddings/) (доступна в библиотеке gensim).

![img](https://amitness.com/images/fasttext-center-word-embedding.png)

## Сжатие моделей

"Хорошие" векторные представления, поскольку обучаются на большом количестве токенов\текстов, довольно большие. Гигабайты данных. Их трудно загружать в Colab.

Мы схитрим, будем использовать сжатые векторные представления, которые может быть похуже, но не сильно, но гораздо меньше!

Подходы к сжатию ([отсюда](https://towardsdatascience.com/compressing-unsupervised-fasttext-models-eb212e9919ca)):
- выбросить большую часть векторов, а хранить только для наиболее часто встречаемых слов и n-грамм.
- хранить вектора с меньшей точностью (float16 вместо float32).
- разбить матрицу векторов на части, кластеризовать, вместо самой части хранить только экземпляр кластера и ID этого кластера.
- факторизовать матрицу векторов произведением меньших матриц, хранить их (сильно теряется точность).

![img](https://d33wubrfki0l68.cloudfront.net/82e81d0b73bce8edc6d180dc80d8f6ecf644bac2/8b910/images/product-quantization-8.jpg)


Такие подходы реализованы в библиотеке `compress-fasttext`, основана на `gensim`, и дают векторные представления в сотни раз меньшие по объему, с несильной потерей точности.  

In [1]:
# ставим библиотеку со сжатыми векторными представлениями
!pip install -q compress-fasttext

  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 2.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.6/60.6 kB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 26.7/26.7 MB 72.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.3/18.3 MB 95.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 38.6/38.6 MB 16.0 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
thinc 8.3.6 requires numpy<3.0.0,>=2.0.0, but you have numpy 1.26.4 which is incompatible.


In [1]:
# подключаем, загружаем сжатые вектора.
import compress_fasttext
wv = compress_fasttext.models.CompressedFastTextKeyedVectors.load(
'https://github.com/avidale/compress-fasttext/releases/download/v0.0.4/cc.en.300.compressed.bin')

In [3]:
#from gensim.models import fasttext
#from gensim.test.utils import datapath

word='polycarbon'
print(word in wv.key_to_index)  # проверяем есть ли такое слово в словаре
print(wv[word].shape)  # вектор для него

word='carbon'
print(word in wv.key_to_index)  # проверяем есть ли такое слово в словаре
print(wv[word].shape)  # вектор для него

word='polycarbonate'
print(word in wv.key_to_index)  # проверяем есть ли такое слово в словаре
print(wv[word].shape)  # вектор для него

False
(300,)
True
(300,)
False
(300,)


In [4]:
wv.similarity('polycarbon','polycarbonate')

0.7487792

In [5]:
# словарь
wv.key_to_index

{',': 0,
 'the': 1,
 '.': 2,
 'and': 3,
 'to': 4,
 'of': 5,
 'a': 6,
 '</s>': 7,
 'in': 8,
 'is': 9,
 ':': 10,
 'I': 11,
 'for': 12,
 'that': 13,
 ')': 14,
 '"': 15,
 '(': 16,
 'on': 17,
 'with': 18,
 'it': 19,
 'you': 20,
 'The': 21,
 'was': 22,
 'as': 23,
 'are': 24,
 'at': 25,
 '/': 26,
 '’': 27,
 'be': 28,
 'by': 29,
 "'s": 30,
 'this': 31,
 'have': 32,
 'from': 33,
 'or': 34,
 '!': 35,
 'not': 36,
 'your': 37,
 'an': 38,
 "'": 39,
 'but': 40,
 '?': 41,
 'can': 42,
 '-': 43,
 'will': 44,
 's': 45,
 'my': 46,
 'has': 47,
 'all': 48,
 'we': 49,
 'they': 50,
 'he': 51,
 'his': 52,
 'more': 53,
 'one': 54,
 'about': 55,
 'their': 56,
 "'t": 57,
 'so': 58,
 'which': 59,
 'It': 60,
 'out': 61,
 'up': 62,
 '...': 63,
 'were': 64,
 'had': 65,
 'who': 66,
 'like': 67,
 ';': 68,
 '“': 69,
 'our': 70,
 'would': 71,
 '”': 72,
 'time': 73,
 'been': 74,
 'if': 75,
 'also': 76,
 'just': 77,
 'when': 78,
 'her': 79,
 'This': 80,
 'me': 81,
 'there': 82,
 'do': 83,
 'what': 84,
 'some': 85,
 'other

In [6]:
# даже для такой ерунды есть вектор
word1='абракадабра111___)))'
print(word1 in wv.key_to_index)  # проверяем есть ли такое слово в словаре
print(wv[word1].shape)  # вектор для него

False
(300,)


In [7]:
# отличие на один символ
word2='абракадабра111___)))F'
print(word2 in wv.key_to_index)  # проверяем есть ли такое слово в словаре
print(wv[word2].shape)  # вектор для него

False
(300,)


In [8]:
# похожесть
wv.similarity(word1,word2)

1.0

In [9]:
wv.similarity('абракадабра111___)))','dfkljfdkjlcvdlksdsdposdf,m;l')

-0.041686714

In [10]:
wv.similarity('решето','решетка')

1.0

In [11]:
# похожие слова
similarities = wv.most_similar(positive=['time'])
print(similarities)

[('day', 0.5973712126522565), ('period', 0.590262867944538), ('moment', 0.5866017938947804), ('last', 0.5650749486972709), ('hours', 0.5359855316309537), ('periods', 0.5333457753961995), ('spend', 0.529741088077883), ('days', 0.5251064244125222), ('year', 0.520560190606691), ('when', 0.5161178628641487)]


In [14]:
# не связанное слово
not_matching = wv.doesnt_match("human computer interface tree".split())
print(not_matching)

tree


In [15]:
# похожесть слов
sim_score = wv.similarity('computer', 'human')
print(sim_score)

0.2725851022412919
